# Stable Diffusion Image Generation with Gradio UI

This notebook allows you to generate images from text prompts using Stable Diffusion. It runs on a free Google Colab GPU and provides a Gradio interface for interaction.

**Instructions:**
1.  **Enable GPU:** Go to `Runtime` -> `Change runtime type` and select `GPU` (e.g., T4) as the hardware accelerator.
2.  **Run Cells:** Execute each cell in order.
    *   The first cell installs necessary libraries.
    *   The second cell imports libraries and defines the image generation function and Gradio UI.
    *   The third cell loads the Stable Diffusion model (this may take a few minutes, especially on the first run as it downloads model weights).
    *   The fourth cell launches the Gradio interface. Click the public URL it provides to open the UI.

In [ ]:
# @title 1. Install Dependencies
!pip install diffusers transformers accelerate gradio bitsandbytes --quiet

## 2. Import Libraries and Define Generation Function & UI

In [ ]:
# @title 2. Import Libraries, Define Generation Function & UI
import gradio as gr
import torch
from diffusers import StableDiffusionPipeline, EulerDiscreteScheduler
from PIL import Image
import random

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# Global variable for the pipeline
pipe = None
model_id = "stabilityai/stable-diffusion-2-1-base" # You can change this to other models like "runwayml/stable-diffusion-v1-5"

def load_model():
    """Loads the Stable Diffusion model."""
    global pipe
    if pipe is None:
        print(f"Loading model: {model_id}...")
        scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
        try:
            pipe = StableDiffusionPipeline.from_pretrained(
                model_id,
                scheduler=scheduler,
                torch_dtype=torch.float16,
                # load_in_8bit=True, # Enable this for lower VRAM usage, requires bitsandbytes
                # device_map="auto"
            )
            pipe = pipe.to("cuda")
            # pipe.enable_attention_slicing() # Enable for further VRAM reduction at a slight performance cost
            print("Model loaded successfully.")
        except Exception as e:
            print(f"Error loading model: {e}")
            pipe = None # Ensure pipe is None if loading failed
    else:
        print("Model already loaded.")

def generate_image(prompt, negative_prompt, seed_value, num_inference_steps=25, guidance_scale=7.5):
    """Generates an image based on the prompt, negative prompt, and seed."""
    if pipe is None:
        return None, "Model not loaded. Please run the model loading cell first."
    
    print(f"Generating image with prompt: '{prompt}'")
    if negative_prompt:
        print(f"Negative prompt: '{negative_prompt}'")
    
    # Use provided seed or generate a random one if empty or invalid
    try:
        seed = int(seed_value)
    except (ValueError, TypeError):
        seed = random.randint(0, 2**32 - 1)
    print(f"Using seed: {seed}")
    
    generator = torch.Generator("cuda").manual_seed(seed)
    
    try:
        with torch.autocast("cuda"):
            image = pipe(
                prompt,
                negative_prompt=negative_prompt if negative_prompt else None,
                num_inference_steps=int(num_inference_steps),
                guidance_scale=guidance_scale,
                generator=generator
            ).images[0]
        print("Image generated.")
        return image, f"Seed used: {seed}"
    except Exception as e:
        print(f"Error during image generation: {e}")
        return None, f"Error: {e}"

# Gradio Interface Definition
with gr.Blocks(css="footer {display: none !important}") as demo:
    gr.Markdown("# Stable Diffusion Image Generator")
    gr.Markdown("Enter a prompt and a negative prompt, then click 'Generate Image'.")
    
    with gr.Row():
        with gr.Column(scale=3):
            prompt_input = gr.Textbox(label="Prompt", placeholder="e.g., A photo of an astronaut riding a horse on the moon")
            negative_prompt_input = gr.Textbox(label="Negative Prompt (Optional)", placeholder="e.g., blurry, low quality, ugly, text, watermark")
            seed_input = gr.Textbox(label="Seed (Optional)", placeholder="Enter a number or leave blank for random")
            with gr.Row():
                 num_steps_slider = gr.Slider(minimum=10, maximum=100, value=25, step=1, label="Number of Inference Steps")
                 guidance_slider = gr.Slider(minimum=1, maximum=20, value=7.5, step=0.1, label="Guidance Scale")
            generate_button = gr.Button("Generate Image", variant="primary")
        with gr.Column(scale=2):
            image_output = gr.Image(label="Generated Image", type="pil", show_download_button=True)
            status_output = gr.Textbox(label="Status", interactive=False)

    generate_button.click(
        generate_image, 
        inputs=[prompt_input, negative_prompt_input, seed_input, num_steps_slider, guidance_slider],
        outputs=[image_output, status_output]
    )
    
    gr.Markdown("## Tips for Better Prompts:
    - Be specific: 'A hyperrealistic 4K photo of a red apple on a wooden table' is better than 'apple'.
    - Use artistic styles: '...in the style of Van Gogh', '...as a watercolor painting', '...cyberpunk art'.
    - Add details: '...with dramatic lighting', '...detailed fur', '...wearing a tiny hat'.
    - Use negative prompts to exclude unwanted elements: 'ugly, blurry, watermark, text, disfigured'.")

print("Gradio UI defined. Next, load the model and then launch the app.")

## 3. Load the Stable Diffusion Model

This cell will download and load the pre-trained Stable Diffusion model. This can take several minutes, especially the first time you run it, as it needs to download the model weights (several gigabytes).

In [ ]:
# @title 3. Load the Model (can take a few minutes)
load_model()

## 4. Launch the Gradio App

Run the cell below to start the Gradio interface. Click the public URL (it usually looks like `https://xxxx.gradio.live`) that appears in the output to open the web UI in a new tab.

In [ ]:
# @title 4. Launch Gradio UI
if pipe is not None:
    demo.launch(debug=True, share=True) # share=True creates a public link
else:
    print("ERROR: Model not loaded. Cannot launch Gradio app. Please ensure the previous cell executed successfully and the model was loaded.")